ETL Datasets

In [4]:
# se importan las librerias que se van a usar 
import os
import gzip
import ast
import pandas as pd
import numpy as np
import nltk

Steam_games

Se abre el archivo json comprimido que contiene la informacion de steam_game

In [5]:
# se define la ruta donde esta ubicado el archivo
file_path = os.path.abspath('../datasets/steam_games.json.gz')
# se lee el archivo json
steam_games = pd.read_json(file_path, compression='gzip', lines=True)
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [6]:
steam_games


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Se identifica las columnas que se van a usar para la creacion de las funciones solicitadas  para los endpoints, por lo cual se procede a hacer la limpieza de la Data. primero se eliminaran las filas que solo contienen valores nulos.

In [7]:
steam_games.dropna(how="all", inplace=True  )
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


Para cambiar el tipo de dato en la columna id se identifico que contiene valores nulos, se procede a verificar.

In [8]:
steam_games[steam_games["id"].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


la fila 88384 solo contiene la url, parece no ser relevante para el analisis por ello se elimina, para la fila 119271 se filtrara para identificar si tiene un duplicado ya que contiene casi toda la informacion y nos puede servir para el analisis posterior

In [9]:
title = "Batman: Arkham City - Game of the Year Edition"
developer = "Rocksteady Studios,Feral Interactive (Mac)"
filtro = steam_games[(steam_games["title"]==title) & (steam_games["developer"]==developer)]
filtro

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


se encontro otra fila con la misma informacion pero con su id, por ello se procede a eliminar la fila que no posee id

In [10]:
steam_games.dropna(subset=['id'], inplace=True) #elimina filas de la columna id que son null

se verifica que la columna id no tenga duplicados

In [11]:
steam_games[steam_games.duplicated(subset=['id'], keep=False)]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


ya que la columna id no deberia contener duplicados se procede a eliminar una de las filas

In [12]:
steam_games.drop_duplicates(subset="id", inplace=True, keep="last")

con los cambios aplicados ya se puede cambiar el tipo de dato

In [13]:
steam_games["id"] = steam_games["id"].astype(int)

la otra columna que se utilizara en las funciones es genres, se procede a verificar

In [14]:
steam_games[steam_games["genres"].isnull()].shape

(3282, 13)

In [15]:
steam_games[(steam_games["genres"].isnull()) & (steam_games["tags"].isnull())].shape

(138, 13)

In [16]:
steam_games.shape

(32132, 13)

Se identifica que la columana genres contiene null esto podria afectar los endpoint... una posible solucion es reemplazar los nulos por los datos de la columna tags ya que en algunas filas el contenido es igual y solo cambia el orden y en otras muy similar.

In [17]:
# Crea un conjunto de géneros únicos 
unique_genres = set(item for val in steam_games['genres'].dropna() for item in val)

# Filtra las etiquetas 'tags' 
steam_games['tags'] = steam_games['tags'].apply(lambda x: [item for item in x if item in unique_genres] if isinstance(x, list) else x) 

# Rellena valores nulos en la columna 'genres' 
steam_games['genres'].fillna(steam_games['tags'], inplace=True)

def add_tags_to_genres(row): 
    ''' Esta función añade valores de 'tags' a 'genres' que no estén ya'''
    genres = row['genres']
    tags = row['tags']
    if isinstance(tags, list) and isinstance(genres, list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres

#aplico la funcion
steam_games['genres'] = steam_games.apply(lambda row: add_tags_to_genres(row), axis=1)

# elimina columna tags
steam_games.drop(columns=['tags'], inplace=True)

In [18]:
steam_games[steam_games["genres"].isnull()].shape

(138, 12)

In [19]:
steam_games.shape

(32132, 12)

In [20]:
steam_games.isnull().sum()

publisher       8051
genres           138
app_name           1
title           2049
url                0
release_date    2066
reviews_url        0
specs            669
price           1377
early_access       0
id                 0
developer       3298
dtype: int64

In [21]:
steam_games.to_csv('../datasets/steam_games.csv',index=False)

User_reviews

Se accede a la informacion del archivo comprimido

In [22]:
file_path_user_reviews = os.path.abspath('../datasets/user_reviews.json.gz')
lista=[]
with gzip.open(file_path_user_reviews, 'rt', encoding='MacRoman') as archivo:
    for line in archivo.readlines():
        lista.append(ast.literal_eval(line))

user_review = pd.DataFrame(lista)
user_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


se procede a desanidar la columna reviews que se necesitara mas adelante pare el proceso 

In [23]:
# Utiliza explode para duplicar las filas generando un diccionario por cada diccionario en la lista
explode_df_user_review = user_review.explode('reviews')

# Utiliza json_normalize para expandir los diccionarios en nuevas columnas
expanded_reviews = pd.json_normalize(explode_df_user_review['reviews'])

# Combina el DataFrame original con el DataFrame generado
explode_df_user_review = explode_df_user_review.drop(["reviews"], axis=1).merge(expanded_reviews, left_index=True, right_index=True)
explode_df_user_review.info()


<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59333 non-null  object
 1   user_url     59333 non-null  object
 2   funny        59314 non-null  object
 3   posted       59314 non-null  object
 4   last_edited  59314 non-null  object
 5   item_id      59314 non-null  object
 6   helpful      59314 non-null  object
 7   recommend    59314 non-null  object
 8   review       59314 non-null  object
dtypes: object(9)
memory usage: 4.5+ MB


In [24]:
explode_df_user_review

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
1,js41637,http://steamcommunity.com/id/js41637,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,Posted June 2.,,209870,0 of 3 people (0%) found this review helpful,True,blackman retribution
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,‡πÄ‡∏£‡∏∑‡πà‡∏≠‡∏á‡∏Ç‡∏≠‡∏á‡∏†‡∏≤‡∏û 10/10 ‡...
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,Posted April 19.,Last edited May 20.,437220,0 of 1 people (0%) found this review helpful,True,‡πÄ‡∏£‡∏∑‡πà‡∏≠‡∏á‡∏Ç‡∏≠‡∏á‡∏†‡∏≤‡∏û 10/10 ‡...


se procede a eliminar las columnas que no son necesarias para el procedimiento

In [25]:
explode_df_user_review.drop(columns=['user_url','funny','helpful','last_edited'],inplace=True)#esta linea elimina las columnas que no necesitamos

Se realiza el analisis de sentimiento solicitado con NLP

In [26]:

from nltk.sentiment import SentimentIntensityAnalyzer

# Descargar recursos de NLTK 
nltk.download('vader_lexicon')

# Crear una instancia de SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Crear una función para asignar el análisis de sentimiento
def assign_sentiment_analysis(review):
    if pd.isna(review):  # Si la reseña está ausente
        return 1  # Valor neutral
    else:
        # Realizar análisis de sentimiento con vaderSentiment
        compound_score = sia.polarity_scores(review)['compound']

        if compound_score >= 0.05:
            return 2  # Positivo
        elif compound_score <= -0.05:
            return 0  # Negativo
        else:
            return 1  # Neutral

# Aplicar la función a la columna 'review' y crear la nueva columna 'sentiment_analysis'
explode_df_user_review['sentiment_analysis'] = explode_df_user_review['review'].apply(assign_sentiment_analysis)

# Reemplazar la columna original 'review' con la nueva 'sentiment_analysis'
explode_df_user_review.drop(columns=['review'], inplace=True)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Willc\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


             user_id                    posted item_id recommend  \
0  76561197970982479  Posted November 5, 2011.    1250      True   
0  76561197970982479  Posted November 5, 2011.    1250      True   
0  76561197970982479  Posted November 5, 2011.    1250      True   
1            js41637     Posted July 15, 2011.   22200      True   
1            js41637     Posted July 15, 2011.   22200      True   

   sentiment_analysis  
0                   2  
0                   2  
0                   2  
1                   2  
1                   2  


In [27]:
explode_df_user_review.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59333 entries, 0 to 25798
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59333 non-null  object
 1   posted              59314 non-null  object
 2   item_id             59314 non-null  object
 3   recommend           59314 non-null  object
 4   sentiment_analysis  59333 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.7+ MB


Se verifica si en el dataframe hay valores nulos

In [36]:
explode_df_user_review.isnull().sum()

user_id               0
posted                0
item_id               0
recommend             0
sentiment_analysis    0
dtype: int64

In [33]:
explode_df_user_review[explode_df_user_review.isnull().any(axis=1)]

,user_id,posted,item_id,recommend,sentiment_analysis
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
137,2xDelorean,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1
177,Mastic,NaN,NaN,NaN,1


ya que las filas estan completamente vacias se procede a eliminarlas

In [35]:
explode_df_user_review.dropna(inplace=True)

Por ultimo despuede de haber realizado las transformaciones solicitadas y las que se consideraron necesarias se procede a exportar a un csv

In [39]:
explode_df_user_review.to_csv('../datasets/user_review.csv',index=False)

User_items

In [14]:

def descomprimir_file_items(ruta, variable_anidada):
    '''Función que recibe una ruta de acceso a un archivo json anidado y carga la información en un
    DataFrame de Pandas'''
    fila = []
    with gzip.open(ruta, 'rt', encoding='MacRoman') as archivo:
        for line in archivo.readlines():
            fila.append(ast.literal_eval(line))

    df = pd.DataFrame(fila)
    df = df.explode(variable_anidada).reset_index(drop=True)
    
    if not df.empty:
        df = pd.concat([df, pd.json_normalize(df[variable_anidada])], axis=1)
        df = df.drop(columns=variable_anidada)

    return df

In [16]:
file_path_items = os.path.abspath("../datasets/users_items.json.gz")

In [17]:
user_items = descomprimir_file_items(file_path_items,'items')

In [18]:
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5170015 entries, 0 to 5170014
Data columns (total 8 columns):
 #   Column            Dtype  
---  ------            -----  
 0   user_id           object 
 1   items_count       int64  
 2   steam_id          object 
 3   user_url          object 
 4   item_id           object 
 5   item_name         object 
 6   playtime_forever  float64
 7   playtime_2weeks   float64
dtypes: float64(2), int64(1), object(5)
memory usage: 315.6+ MB


In [34]:
user_items

,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,10,Counter-Strike,6.0,0.0
1,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,20,Team Fortress Classic,0.0,0.0
2,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,30,Day of Defeat,7.0,0.0
3,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,40,Deathmatch Classic,0.0,0.0
4,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,50,Half-Life: Opposing Force,0.0,0.0
...,...,...,...,...,...,...,...,...
5170010,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,373330,All Is Dust,0.0,0.0
5170011,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,388490,One Way To Die: Steam Edition,3.0,3.0
5170012,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,521570,You Have 10 Seconds 2,4.0,4.0
5170013,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,519140,Minds Eyes,3.0,3.0


In [31]:
user_items.to_csv('../datasets/user_items.csv',index=False)